# Intro to Assign Scores

The `assign_scores()` method is a powerful feature that allows you to compute and add scorer scores as new columns in your dataset. This method takes a model and metric(s) as input, computes the specified metrics from the ValidMind scorer library, and adds them as new columns. The computed metrics provide per-row values, giving you granular insights into model performance at the individual prediction level.

In this interactive notebook, we demonstrate how to use the `assign_scores()` method effectively. We'll walk through a complete example using a customer churn dataset, showing how to compute and assign row-level metrics (like Brier Score and Log Loss) that provide detailed performance insights for each prediction. You'll learn how to work with single and multiple scorers, pass custom parameters, and handle different metric types - all while maintaining a clean, organized dataset structure. Currently, assign_scores() supports all metrics available in the validmind.scorer module.

**The Power of Row-Level Scoring**

Traditional model evaluation workflows often focus on aggregate metrics that provide overall performance summaries. The `assign_scores()` method complements this by providing granular, row-level insights that help you:

- **Identify Problematic Predictions**: Spot individual cases where your model performs poorly
- **Understand Model Behavior**: Analyze how model performance varies across different types of inputs
- **Enable Detailed Analysis**: Perform targeted investigations on specific subsets of your data
- **Support Model Debugging**: Pinpoint exactly where and why your model makes errors

**Understanding assign_scores()**

The `assign_scores()` method computes row metrics for a given model-dataset combination and adds the results as new columns to your dataset. Each new column follows the naming convention: `{model.input_id}_{metric_name}`, ensuring clear identification of which model and metric combination generated each score.

Key features:

- **Row-Level Focus**: Computes per-prediction metrics rather than aggregate scores
- **Flexible Input**: Accepts single metrics or lists of metrics
- **Parameter Support**: Allows passing additional parameters to underlying metric implementations
- **Multi-Model Support**: Can assign scores from multiple models to the same dataset
- **Type Agnostic**: Works with classification, regression, and other model types

This approach provides detailed insights into your model's performance at the individual prediction level, enabling more sophisticated analysis and debugging workflows.

::: {.content-hidden when-format="html"}
## Contents    
- [About ValidMind](#toc1__)    
  - [Before you begin](#toc1_1__)    
  - [New to ValidMind?](#toc1_2__)    
- [Setting up](#toc2__)    
  - [Install the ValidMind Library](#toc2_1__)    
  - [Initialize the ValidMind Library](#toc2_2__)    
    - [Register sample model](#toc2_2_1__)    
    - [Apply documentation template](#toc2_2_2__)    
    - [Get your code snippet](#toc2_2_3__)    
- [Load the demo dataset](#toc3__)    
- [Train models for testing](#toc4__)    
- [Initialize ValidMind objects](#toc5__)    
- [Assign predictions](#toc6__)    
- [Using assign_scores()](#toc7__)    
  - [Basic Usage](#toc7_1__)    
  - [Single Scorer Assignment](#toc7_2__)    
  - [A Scorer returns complex object](#toc7_3__)    
  - [Multiple Scorers Assignment](#toc7_4__)    
  - [Passing Parameters to Scorer](#toc7_5__)    
  - [Multi-Model scorers](#toc7_6__)    
  - [Scorer Metrics](#toc7_7__)    
  - [Custom Scorer](#toc7_8__)    
- [Next steps](#toc8__)    
  - [Work with your model documentation](#toc8_1__)    
  - [Discover more learning resources](#toc8_2__)    
- [Upgrade ValidMind](#toc9__)    

:::
<!-- jn-toc-notebook-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=4
	/jn-toc-notebook-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

<a id='toc1__'></a>

## About ValidMind

ValidMind is a suite of tools for managing model risk, including risk associated with AI and statistical models.

You use the ValidMind Library to automate documentation and validation tests, and then use the ValidMind Platform to collaborate on model documentation. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

<a id='toc1_1__'></a>

### Before you begin

This notebook assumes you have basic familiarity with Python, including an understanding of how functions work. If you are new to Python, you can still run the notebook but we recommend further familiarizing yourself with the language. 

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

<a id='toc1_2__'></a>

### New to ValidMind?

If you haven't already seen our documentation on the [ValidMind Library](https://docs.validmind.ai/developer/validmind-library.html), we recommend you begin by exploring the available resources in this section. There, you can learn more about documenting models and running tests, as well as find code samples and our Python Library API reference.

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>For access to all features available in this notebook, you'll need access to a ValidMind account.</b></span>
<br></br>
<a href="https://docs.validmind.ai/guide/configuration/register-with-validmind.html" style="color: #DE257E;"><b>Register with ValidMind</b></a></div>

<a id='toc2__'></a>

## Setting up

<a id='toc2_1__'></a>

### Install the ValidMind Library

To install the library:

In [ ]:
%pip install -q validmind

<a id='toc2_2__'></a>

### Initialize the ValidMind Library

<a id='toc2_2_1__'></a>

#### Register sample model

Let's first register a sample model for use with this notebook:

1. In a browser, [log in to ValidMind](https://docs.validmind.ai/guide/configuration/log-in-to-validmind.html).

2. In the left sidebar, navigate to **Inventory** and click **+ Register Model**.

3. Enter the model details and click **Next >** to continue to assignment of model stakeholders. ([Need more help?](https://docs.validmind.ai/guide/model-inventory/register-models-in-inventory.html))

4. Select your own name under the **MODEL OWNER** drop-down.

5. Click **Register Model** to add the model to your inventory.

<a id='toc2_2_2__'></a>

#### Apply documentation template

Once you've registered your model, let's select a documentation template. A template predefines sections for your model documentation and provides a general outline to follow, making the documentation process much easier.

1. In the left sidebar that appears for your model, click **Documents** and select **Documentation**.

2. Under **TEMPLATE**, select `Binary classification`.

3. Click **Use Template** to apply the template.

<a id='toc2_2_3__'></a>

#### Get your code snippet

ValidMind generates a unique _code snippet_ for each registered model to connect with your developer environment. You initialize the ValidMind Library with this code snippet, which ensures that your documentation and tests are uploaded to the correct model when you run the notebook.

1. On the left sidebar that appears for your model, select **Getting Started** and click **Copy snippet to clipboard**.
2. Next, [load your model identifier credentials from an `.env` file](https://docs.validmind.ai/developer/model-documentation/store-credentials-in-env-file.html) or replace the placeholder with your own code snippet:

In [ ]:
# Load your model identifier credentials from an `.env` file

%load_ext dotenv
%dotenv .env

# Or replace with your code snippet

import validmind as vm

vm.init(
    api_host="...",
    api_key="...",
    api_secret="...",
    model="...",
)


<a id='toc3__'></a>

## Load the demo dataset

In this example, we load a demo dataset to demonstrate the assign_scores functionality with customer churn prediction models.

In [ ]:
from validmind.datasets.classification import customer_churn as demo_dataset

print(
    f"Loaded demo dataset with: \n\n\t• Target column: '{demo_dataset.target_column}' \n\t• Class labels: {demo_dataset.class_labels}"
)

raw_df = demo_dataset.load_data()
raw_df.head()


<a id='toc4__'></a>

## Train models for testing

We'll train two different customer churn models to demonstrate the assign_scores functionality with multiple models.

In [ ]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier

# Preprocess the data
train_df, validation_df, test_df = demo_dataset.preprocess(raw_df)

# Prepare training data
x_train = train_df.drop(demo_dataset.target_column, axis=1)
y_train = train_df[demo_dataset.target_column]
x_val = validation_df.drop(demo_dataset.target_column, axis=1)
y_val = validation_df[demo_dataset.target_column]

# Train XGBoost model
xgb_model = xgb.XGBClassifier(early_stopping_rounds=10, random_state=42)
xgb_model.set_params(
    eval_metric=["error", "logloss", "auc"],
)
xgb_model.fit(
    x_train,
    y_train,
    eval_set=[(x_val, y_val)],
    verbose=False,
)

# Train Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train, y_train)

print("Models trained successfully!")
print(f"XGBoost training accuracy: {xgb_model.score(x_train, y_train):.3f}")
print(f"Random Forest training accuracy: {rf_model.score(x_train, y_train):.3f}")


<a id='toc5__'></a>

## Initialize ValidMind objects

We initialize ValidMind `dataset` and `model` objects. The `input_id` parameter is crucial for the assign_scores functionality as it determines the column naming convention for assigned scores.

In [ ]:
# Initialize datasets
vm_train_ds = vm.init_dataset(
    input_id="train_dataset",
    dataset=train_df,
    target_column=demo_dataset.target_column,
)
vm_test_ds = vm.init_dataset(
    input_id="test_dataset",
    dataset=test_df,
    target_column=demo_dataset.target_column,
)

# Initialize models with descriptive input_ids
vm_xgb_model = vm.init_model(model=xgb_model, input_id="xgboost_model")
vm_rf_model = vm.init_model(model=rf_model, input_id="random_forest_model")

print("ValidMind objects initialized successfully!")
print(f"XGBoost model ID: {vm_xgb_model.input_id}")
print(f"Random Forest model ID: {vm_rf_model.input_id}")


<a id='toc6__'></a>

## Assign predictions

Before we can use assign_scores(), we need to assign predictions to our datasets. This step is essential as many unit metrics require both actual and predicted values.

In [ ]:
# Assign predictions for both models to both datasets
vm_train_ds.assign_predictions(model=vm_xgb_model)
vm_train_ds.assign_predictions(model=vm_rf_model)

vm_test_ds.assign_predictions(model=vm_xgb_model)
vm_test_ds.assign_predictions(model=vm_rf_model)

print("Predictions assigned successfully!")
print(f"Test dataset now has {len(vm_test_ds.df.columns)} columns")


<a id='toc7__'></a>

## Using assign_scores()

Now we'll explore the various ways to use the assign_scores() method to integrate performance metrics directly into your dataset.

<a id='toc7_1__'></a>

### Basic Usage

The assign_scores() method has a simple interface:

```python
dataset.assign_scores(model, metrics, **kwargs)
```

- **model**: A ValidMind model object
- **metrics**: Single metric ID or list of metric IDs (can use short names or full IDs)
- **kwargs**: Additional parameters passed to the underlying metric implementations

Let's first check what columns we currently have in our test dataset:

In [ ]:
print("Current columns in test dataset:")
for i, col in enumerate(vm_test_ds.df.columns, 1):
    print(f"{i:2d}. {col}")

print(f"\nDataset shape: {vm_test_ds.df.shape}")

<a id='toc7_2__'></a>

### Single Scorer Assignment
 
Let's start by assigning a single Scorer - the Brier Score - for our XGBoost model on the test dataset.

In [ ]:
# Assign Brier Score for XGBoost model
vm_test_ds.assign_scores(metrics = "validmind.scorers.classification.BrierScore", model = vm_xgb_model)

print("After assigning Brier Score:")
print(f"New column added: {vm_test_ds.df.columns}")
# Display the metric values
vm_test_ds.df.head()

<a id='toc7_3__'></a>

### A Scorer returns complex object
 The OutlierScore scorer demonstrates how scorers can return complex objects. It returns a dictionary containing per-row outlier detection results. For each row, it includes:
 - is_outlier: Boolean indicating if the row is an outlier
 - anomaly_score: Numerical score indicating degree of outlierness
 - isolation_path: Length of isolation path in the tree

When assigned to a dataset, these dictionary values are automatically unpacked into separate columns with appropriate prefixes.

In [ ]:
# Assign Brier Score for XGBoost model
vm_test_ds.assign_scores(metrics = "validmind.scorers.classification.OutlierScore", model = vm_xgb_model)

print("After assigning Score With Confidence:")
print(f"New column added: {vm_test_ds.df.columns}")
# Display the metric values
vm_test_ds.df.head()

In [ ]:
# Assign Brier Score for XGBoost model
vm_test_ds.assign_scores("validmind.scorers.classification.OutlierScore")

print("After assigning Score With Confidence:")
print(f"New column added: {vm_test_ds.df.columns}")
# Display the metric values
vm_test_ds.df.head()

<a id='toc7_4__'></a>

### Multiple Scorers Assignment

We can assign multiple metrics at once by passing a list of Scorer names. This is more efficient than calling assign_scores() multiple times.

In [ ]:
# Assign multiple classification metrics for the Random Forest model
scorer = [
    "validmind.scorers.classification.BrierScore",
    "validmind.scorers.classification.LogLoss",
    "validmind.scorers.classification.Confidence"
]

vm_test_ds.assign_scores(metrics = scorer, model = vm_rf_model)

print("After assigning multiple row metrics for Random Forest:")
rf_columns = [col for col in vm_test_ds.df.columns if 'random_forest_model' in col]
print(f"Random Forest columns: {rf_columns}")

# Display the metric values
vm_test_ds.df[rf_columns].head()


<a id='toc7_5__'></a>

### Passing Parameters to Scorer

Many row metrics accept additional parameters that are passed through to the underlying implementations. Let's demonstrate this with the LogLoss metric.

In [ ]:
# Assign LogLoss
vm_test_ds.assign_scores(metrics = "validmind.scorers.classification.LogLoss", model = vm_xgb_model, eps = 1e-16)

# We can also assign with different parameters by calling assign_scores again
# Note: This will overwrite the previous column with the same name
print("LogLoss assigned successfully")

# Let's also assign BrierScore and Confidence
vm_test_ds.assign_scores(metrics = ["validmind.scorers.classification.BrierScore","validmind.scorers.classification.Confidence"], model = vm_xgb_model)

print("BrierScore and Confidence assigned successfully")

# Display current XGBoost metric columns
xgb_columns = [col for col in vm_test_ds.df.columns if 'xgboost_model' in col]
print(f"\nXGBoost model columns: {xgb_columns}")

vm_test_ds.df[xgb_columns].head()


<a id='toc7_6__'></a>

### Multi-Model scorers

One of the powerful features of assign_scores() is the ability to assign scores from multiple models to the same dataset, enabling detailed model comparison at the prediction level.

In [ ]:
# Let's assign a comprehensive set of metrics for both models
comprehensive_metrics = [
    "validmind.scorers.classification.BrierScore",
    "validmind.scorers.classification.LogLoss",
    "validmind.scorers.classification.Confidence",
    "validmind.scorers.classification.Correctness"
]

# Assign for XGBoost model
vm_test_ds.assign_scores(metrics = comprehensive_metrics, model = vm_xgb_model)

# Assign for Random Forest model}
vm_test_ds.assign_scores(metrics = comprehensive_metrics, model = vm_rf_model)

print("Row-level metrics assigned for both models!")


<a id='toc7_7__'></a>

### Scorer Metrics
The next section demonstrates how to assign individual metrics that compute scores per row, rather than aggregate metrics.
We'll use several important row metrics:
 
- Brier Score: Measures how well calibrated the model's probability predictions are for each individual prediction
- Log Loss: Evaluates how well the predicted probabilities match the true labels on a per-prediction basis
- Confidence: Measures the model's confidence in its predictions for each row
- Correctness: Indicates whether each prediction is correct (1) or incorrect (0)

All these metrics provide granular insights into model performance at the individual prediction level.

In [ ]:
# Let's add some individual metrics that compute per-row scores
print("Adding individual metrics...")

# Add Brier Score - measures accuracy of probabilistic predictions per row
vm_test_ds.assign_scores(metrics = "validmind.scorers.classification.BrierScore", model = vm_xgb_model)
print("Added Brier Score - lower values indicate better calibrated probabilities")

# Add Log Loss - measures how well the predicted probabilities match true labels per row
vm_test_ds.assign_scores(metrics = "validmind.scorers.classification.LogLoss", model = vm_xgb_model)
print("Added Log Loss - lower values indicate better probability estimates")

# Create a comparison summary showing first few rows of individual metrics
print("\nFirst few rows of individual metrics:")
individual_metrics = [col for col in vm_test_ds.df.columns if any(m in col for m in ['BrierScore', 'LogLoss', 'Confidence', 'Correctness'])]
print(vm_test_ds.df[individual_metrics].head())


In [ ]:
vm_test_ds._df.head()

<a id='toc7_8__'></a>

### Custom Scorer
Let's see how to create your own custom scorers using the `@scorer` decorator.
 
The example below demonstrates a scorer that looks at the class balance in the neighborhood around each data point. For each row, it will give you a score from 0 to 1, where a score closer to 1 means there's a nice even balance of classes in that area of your data. This can help you identify regions where your classes are well-mixed vs regions dominated by a single class.

In [ ]:
from validmind.scorer import scorer
import numpy as np

@scorer("my_scorers.TestScorer") 
def test_scorer(model, dataset):
    """Custom scorer that calculates class balance ratio.
    
    Args:
        model: Not used in this scorer
        dataset: The dataset to analyze
        
    Returns:
        numpy.ndarray: Array of class balance ratios between 0 and 1,
        where values closer to 1 indicate better class balance in the local neighborhood
    """
    # Get target values
    y = dataset.df[dataset.target_column].values
    
    # Calculate local class balance in sliding windows
    window_size = 100
    balance_scores = []
    
    for i in range(len(y)):
        start_idx = max(0, i - window_size//2)
        end_idx = min(len(y), i + window_size//2)
        window = y[start_idx:end_idx]
        
        # Calculate ratio of minority class
        class_ratio = np.mean(window)
        # Adjust to be symmetric around 0.5
        balance_score = 1 - abs(0.5 - class_ratio) * 2
        
        balance_scores.append(balance_score)
        
    return np.array(balance_scores)

# Assign the class balance scores to the dataset
vm_test_ds.assign_scores(metrics = "my_scorers.TestScorer", model = vm_xgb_model)
    

<a id='toc8__'></a>

## Next steps

You can explore the assigned scores right in the notebook as demonstrated above. However, there's even more value in using the ValidMind Platform to work with your model documentation and monitoring.

<a id='toc8_1__'></a>

### Work with your model documentation

1. From the **Model Inventory** in the ValidMind Platform, go to the model you registered earlier. ([Need more help?](https://docs.validmind.ai/guide/model-inventory/working-with-model-inventory.html))

2. Click and expand the **Model Development** section.

The scores you've assigned using `assign_scores()` become part of your model's documentation and can be used in ongoing monitoring workflows. You can view these metrics over time, set up alerts for performance drift, and compare models systematically. [Learn more ...](https://docs.validmind.ai/guide/model-documentation/working-with-model-documentation.html)

<a id='toc8_2__'></a>

### Discover more learning resources

We offer many interactive notebooks to help you work with model scoring and evaluation:

- [Run unit metrics](https://docs.validmind.ai/developer/model-testing/testing-overview.html)
- [Assign predictions](https://docs.validmind.ai/developer/samples-jupyter-notebooks.html)
- [Model comparison workflows](https://docs.validmind.ai/developer/samples-jupyter-notebooks.html)

Or, visit our [documentation](https://docs.validmind.ai/) to learn more about ValidMind.

<a id='toc9__'></a>

## Upgrade ValidMind

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;">After installing ValidMind, you'll want to periodically make sure you are on the latest version to access any new features and other enhancements.</div>

Retrieve the information for the currently installed version of ValidMind:

%pip show validmind

If the version returned is lower than the version indicated in our [production open-source code](https://github.com/validmind/validmind-library/blob/prod/validmind/__version__.py), restart your notebook and run:

```bash
%pip install --upgrade validmind
```

You may need to restart your kernel after running the upgrade package for changes to be applied.

<!-- VALIDMIND COPYRIGHT -->

<small>

***

Copyright © 2023-2026 ValidMind Inc. All rights reserved.<br>
Refer to [LICENSE](https://github.com/validmind/validmind-library/blob/main/LICENSE) for details.<br>
SPDX-License-Identifier: AGPL-3.0 AND ValidMind Commercial</small>